In [1]:
from random import randrange, random
alphabet = list('abcdefghijklmnopqrstuvwxyz ') # space included, all text assumed to be lowercase

In [2]:
def get_relative_letter_frequencies(text):
    counts = {}
    for letter in alphabet:
        counts[letter] = 0
        
    total_count = 0
    for letter in text:
        if letter in alphabet:
            counts[letter] += 1
            total_count += 1
    
    # convert counts to relative frequencies
    for letter in alphabet:
        counts[letter] /= total_count
    return counts

def get_letter_transition_matrix(text):
    transition_counts = {}
    for l1 in alphabet:
        transition_counts[l1] = {}
        for l2 in alphabet:
            transition_counts[l1][l2] = 1
    
    row_counts = {}
    for letter in alphabet:
        row_counts[letter] = len(alphabet)

    previous_letter = ''
    for letter in text:
        if (letter in alphabet) and (previous_letter in alphabet):
            transition_counts[previous_letter][letter] += 1
            row_counts[previous_letter] += 1
        previous_letter = letter
    
    # convert counts to relative frequencies
    for l1 in alphabet:
        for l2 in alphabet:
            transition_counts[l1][l2] /= row_counts[l1]
    return transition_counts

In [5]:
import requests
url = 'http://www.gutenberg.org/files/2600/2600-0.txt' # War and Peace text
text = requests.get(url).text.lower()
letter_frequencies = get_relative_letter_frequencies(text)
letter_transitions = get_letter_transition_matrix(text)

In [6]:
def get_random_cipher():
    remaining_letters = alphabet[:] # create a copy of the alphabet
    cipher = {}
    for letter in alphabet:
        cipher[letter] = remaining_letters.pop(randrange(len(remaining_letters)))
    return cipher

In [7]:
def get_cipher_guess(enciphered_message, letter_frequencies):
    message_letter_frequencies = {}
    for letter in alphabet:
        message_letter_frequencies[letter] = 0
    for letter in enciphered_message:
        message_letter_frequencies[letter] += 1
    
    sorted_message_letters = sorted(message_letter_frequencies, key=message_letter_frequencies.get)
    sorted_letters = sorted(letter_frequencies, key = letter_frequencies.get)
    cipher = {}
    for i in range(len(alphabet)):
        cipher[sorted_message_letters[i]] = sorted_letters[i]
    return cipher

In [8]:
def get_cipher_plausibility(cipher, enciphered_message, letter_transitions):
    plausibility = 1
    previous_letter = ''
    for letter in enciphered_message:
        if (letter in alphabet) and (previous_letter in alphabet):
            plausibility *= letter_transitions[cipher[previous_letter]][cipher[letter]]
        previous_letter = letter
    return plausibility

In [9]:
def get_similar_cipher(cipher): # make random transposition
    alphabet_copy = alphabet[:]
    letter1 = alphabet_copy.pop(randrange(len(alphabet_copy)))
    letter2 = alphabet_copy[randrange(len(alphabet_copy))]
    
    new_cipher = cipher.copy()
    temp = cipher[letter1]
    new_cipher[letter1] = cipher[letter2]
    new_cipher[letter2] = temp
    return new_cipher

In [10]:
def metropolis_hastings_step(enciphered_message, letter_transitions, initial_cipher, num_iterations):
    cipher = initial_cipher
    for i in range(num_iterations):
        plausibility = get_cipher_plausibility(cipher, enciphered_message, letter_transitions)
        new_cipher = get_similar_cipher(cipher)
        new_plausibility = get_cipher_plausibility(new_cipher, enciphered_message, letter_transitions)
        if new_plausibility > plausibility or plausibility == 0:
            cipher = new_cipher
        else:
            alpha = new_plausibility / plausibility
            if random() < alpha:
                cipher = new_cipher
    return cipher

In [11]:
def decipher_message(enciphered_message, cipher):
    return ''.join([cipher[letter] for letter in enciphered_message])

In [14]:
def metropolis_hastings(enciphered_message, letter_transitions, letter_frequencies, step_size, num_steps):
    cipher = get_cipher_guess(enciphered_message, letter_frequencies)
    print(decipher_message(enciphered_message, cipher))
    for i in range(num_steps):
        cipher = metropolis_hastings_step(enciphered_message, letter_transitions, cipher, step_size)
        print(str((i+1)*step_size) + '/' + str(num_steps*step_size) + ': ' + decipher_message(enciphered_message, cipher))

In [15]:
message = 'enter hamlet ham to be or not to be that is the question whether tis nobler in the mind to suffer the slings and arrows of outrageous fortune or to take arms against a sea of troubles and by opposing end'
cipher = get_random_cipher()
enciphered_message = decipher_message(message, cipher) # deciphering w/ random cipher functions as enciphering
metropolis_hastings(enciphered_message, letter_transitions, letter_frequencies, step_size=1000, num_steps=20)

tneth somwte som ea ut ah nae ea ut esoe ri est vdtieran gstesth eri nauwth rn est mrnc ea idllth est iwrnfi onc ohhagi al adehoftadi lahednt ah ea eobt ohmi ofornie o ito al ehaduwti onc up ayyairnf tnc
1000/20000: ensed hoftes hof sa pe ad nas sa pe shos ur she xiersuan cheshed sur napted un she fung sa rimmed she rtunvr ong oddacr am aisdoveair madsine ad sa sowe odfr ovounrs o reo am sdaipter ong py allarunv eng
2000/20000: enter holket hol ta ce ar nat ta ce thot is the puestian whether tis nacker in the ling ta summer the skinds ong orraws am autrodeaus martune ar ta tove orls odoinst o seo am trauckes ong cy affasind eng
3000/20000: enter halket hal to ce or not to ce that is the buestion whether tis nocker in the ling to suffer the skinds ang arrows of outradeous fortune or to tave arls adainst a sea of trouckes ang cy ommosind eng
4000/20000: enter halket hal to ce or not to ce that is the buestion whether tis nocker in the lind to summer the skings and arrows om outrageous mo